In [ ]:
SYFT_VERSION = ">=0.9,<1.0.0"
package_string = f'"syft{SYFT_VERSION}"'
# %pip install {package_string} -q

In [ ]:
# third party
import numpy as np

# syft absolute
import syft as sy

sy.requires(SYFT_VERSION)

In [ ]:
server = sy.orchestra.launch(name="test-datasite-1", dev_mode=True, reset=True)

In [ ]:
datasite_client = server.login(email="info@openmined.org", password="changethis")

In [ ]:
datasite_client.register(
    email="newuser@openmined.org", name="John Doe", password="pw", password_verify="pw"
)

In [ ]:
client_low_ds = server.login(email="newuser@openmined.org", password="pw")

In [ ]:
# stdlib
from typing import Any

In [ ]:
# syft absolute
import syft as sy


class RepeatedCallPolicy(sy.CustomOutputPolicy):
    n_calls: int = 0
    downloadable_output_args: list[str] = []
    state: dict[Any, Any] = {}

    def __init__(self, n_calls=1, downloadable_output_args: list[str] = None):
        self.downloadable_output_args = (
            downloadable_output_args if downloadable_output_args is not None else []
        )
        self.n_calls = n_calls
        self.state = {"counts": 0}

    def public_state(self):
        return self.state["counts"]

    def update_policy(self, context, outputs):
        self.state["counts"] += 1

    def apply_to_output(self, context, outputs, update_policy=True):
        if hasattr(outputs, "syft_action_data"):
            outputs = outputs.syft_action_data
        output_dict = {}
        if self.state["counts"] < self.n_calls:
            for output_arg in self.downloadable_output_args:
                output_dict[output_arg] = outputs[output_arg]
            if update_policy:
                self.update_policy(context, outputs)
        else:
            return None
        return output_dict

    def is_valid(self, context):
        return self.state["counts"] < self.n_calls

In [ ]:
policy = RepeatedCallPolicy(n_calls=1, downloadable_output_args=["y"])

In [ ]:
policy.n_calls

In [ ]:
policy.downloadable_output_args

In [ ]:
policy.init_kwargs

In [ ]:
print(policy.init_kwargs)
a_obj = sy.ActionObject.from_obj({"y": [1, 2, 3]})
x = policy.apply_to_output(None, a_obj)
x["y"]

In [ ]:
policy.n_calls

In [ ]:
x = np.array([1, 2, 3])
x_pointer = sy.ActionObject.from_obj(x)
x_pointer

In [ ]:
x_pointer = x_pointer.send(datasite_client)

In [ ]:
# third party

# syft absolute
from syft import SyftException
from syft import UID
from syft.client.api import AuthedServiceContext
from syft.client.api import ServerIdentity


class CustomExactMatch(sy.CustomInputPolicy):
    def __init__(self, *args: Any, **kwargs: Any) -> None:
        pass

    def filter_kwargs(self, kwargs, context):  # stdlib
        allowed_inputs = self.allowed_ids_only(
            allowed_inputs=self.inputs, kwargs=kwargs, context=context
        )
        results = self.retrieve_from_db(
            allowed_inputs=allowed_inputs,
            context=context,
        )
        return results

    def retrieve_from_db(self, allowed_inputs, context):
        # syft absolute
        from syft import ServerType
        from syft.service.action.action_object import TwinMode

        action_service = context.server.get_service("actionservice")
        code_inputs = {}

        # When we are retrieving the code from the database, we need to use the server's
        # verify key as the credentials. This is because when we approve the code, we
        # we allow the private data to be used only for this specific code.
        # but we are not modifying the permissions of the private data

        root_context = AuthedServiceContext(
            server=context.server, credentials=context.server.verify_key
        )
        if context.server.server_type != ServerType.DATASITE:
            raise SyftException(
                public_message=f"Invalid server type for code submission: {context.server.server_type}"
            )

        for var_name, arg_id in allowed_inputs.items():
            code_inputs[var_name] = action_service.get(
                context=root_context,
                uid=arg_id,
                twin_mode=TwinMode.NONE,
            )
        return code_inputs

    def allowed_ids_only(
        self,
        allowed_inputs,
        kwargs,
        context,
    ):
        # syft absolute
        from syft import ServerType

        if context.server.server_type != ServerType.DATASITE:
            raise SyftException(
                public_message=f"Invalid server type for code submission: {context.server.server_type}"
            )

        server_identity = ServerIdentity(
            server_name=context.server.name,
            server_id=context.server.id,
            verify_key=context.server.signing_key.verify_key,
        )
        allowed_inputs = allowed_inputs.get(server_identity, {})

        filtered_kwargs = {}
        for key in allowed_inputs.keys():
            if key in kwargs:
                value = kwargs[key]
                uid = value

                if not isinstance(uid, UID):
                    uid = getattr(value, "id", None)

                if uid != allowed_inputs[key]:
                    raise SyftException(
                        public_message=f"Input with uid: {uid} for `{key}` not in allowed inputs: {allowed_inputs}"
                    )

                filtered_kwargs[key] = value

        return filtered_kwargs

    def is_valid(
        self,
        context,
        usr_input_kwargs,
    ):
        filtered_input_kwargs = self.filter_kwargs(
            kwargs=usr_input_kwargs,
            context=context,
        )

        expected_input_kwargs = set()
        for _inp_kwargs in self.inputs.values():
            for k in _inp_kwargs.keys():
                if k not in usr_input_kwargs:
                    raise SyftException(
                        public_message=f"Function missing required keyword argument: '{k}'"
                    )
            expected_input_kwargs.update(_inp_kwargs.keys())

        permitted_input_kwargs = list(filtered_input_kwargs.keys())

        not_approved_kwargs = set(expected_input_kwargs) - set(permitted_input_kwargs)
        if len(not_approved_kwargs) > 0:
            raise SyftException(
                public_message=f"Function arguments: {not_approved_kwargs} are not approved yet."
            )

        return True

In [ ]:
@sy.syft_function(
    input_policy=CustomExactMatch(x=x_pointer),
    output_policy=RepeatedCallPolicy(n_calls=10, downloadable_output_args=["y"]),
)
def func(x):
    return {"y": x + 1}

In [ ]:
x = CustomExactMatch(x=x_pointer)

In [ ]:
request = client_low_ds.code.request_code_execution(func)
request

In [ ]:
request_id = request.id

In [ ]:
client_low_ds.code.get_all()

In [ ]:
for request in datasite_client.requests:
    if request.id == request_id:
        break

In [ ]:
func = request.code

In [ ]:
# Custom policies need to be approved before they can be viewed and used
assert func.input_policy is None
assert func.output_policy is None

In [ ]:
result = func.run(x=x_pointer)
result

In [ ]:
request.approve()

In [ ]:
assert func.input_policy is not None

In [ ]:
assert func.output_policy is not None

In [ ]:
res_ptr = client_low_ds.code.func(x=x_pointer)
res_ptr

In [ ]:
res = res_ptr.get()
res

In [ ]:
assert (res["y"] == np.array([2, 3, 4])).all()

In [ ]:
assert set(res.keys()) == set("y")

In [ ]:
for code in datasite_client.code.get_all():
    if code.service_func_name == "func":
        break
print(code.output_policy.state)
assert code.output_policy.state == {"counts": 1}

In [ ]:
if server.server_type.value == "python":
    server.land()